# Recommendation Systems for movies

Two MovieLens datasets are used here. (link : https://www.kaggle.com/rounakbanik/the-movies-dataset/data)

* **The Full Dataset:** Consists of 26,000,000 ratings and 750,000 tag applications applied to 45,000 movies by 270,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags.
* **The Small Dataset:** Comprises of 100,000 ratings and 1,300 tag applications applied to 9,000 movies by 700 users.

We will build our Simple Recommender using movies from the *Full Dataset* whereas all personalized recommender systems will make use of the small dataset (due to the computing power I possess being very limited). As a first step, let us build our simple recommender system.

In [49]:
#importing required libraries 
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate



import warnings; warnings.simplefilter('ignore')

In [2]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

## Simple Popularity recommendation model

The model simply do implement generalize recommendation to all user based on popularity of movie. The logic behind the model is average audience probably pick or like movies which is critically acclaimed and most popular.This model does not give personalized recommendations based on the user. 

Procedure is as follow.

- First sort movies based on ratings and popularity
- Display the top movies of list. 

** ---- added step ------- ** <br> 
also used genre argument for the top movies of a particular genre. 

In [50]:
m_df = pd.read_csv('data/movies_metadata.csv')
m_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",10/30/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",12/15/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",12/22/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",12/22/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/10/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [51]:
m_df['genres'] = m_df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] 
                                                                       if isinstance(x, list) else [])

#### IMDB's *weighted rating* formula 

Mathematically, it is represented as follows:

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

where,
* *v* is the number of votes for the movie
* *m* is the minimum votes required to be listed in the chart
* *R* is the average rating of the movie
* *C* is the mean vote across the whole report

Now, determine an cutoff value for *m*, the minimum votes required to be listed in the chart. 

in this model used cutoff is **95th percentile** that means for a movie to feature in the top N must have more votes than at least 95% of the movies in the list.

#### Calculate C

In [52]:
# Claculation of c 
vote_counts = m_df[m_df['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = m_df[m_df['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

#### Calculate m 

In [53]:
## Claculation of m
m = vote_counts.quantile(0.95)
m

434.0

In [54]:
# extracting release year from release_date
m_df['year'] = pd.to_datetime(m_df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] 
                                                                           if x != np.nan else np.nan)

#### Getting qualified movies (cutoff: 95%)

In [55]:
col_list = ['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']
qualified = m_df[(m_df['vote_count'] >= m) 
                 & (m_df['vote_count'].notnull()) 
                 & (m_df['vote_average'].notnull())][col_list]

# converting vote_count and vote_average colums to integer
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

Findings as follow:
1. for consideration in the chart, a movie has to have at least **434 votes**. 
2. the average rating for a movie is **5.244** on a scale of 10. 
3. **2274** Movies qualify to be on the chart.

### Calculating weighted ratings and making top 250 chart

In [56]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [57]:
qualified['weighted_rating'] = qualified.apply(weighted_rating, axis=1)

In [58]:
qualified = qualified.sort_values('weighted_rating', ascending=False).head(250)

### Top 10 Movies

In [59]:
qualified.head(10)

,title,year,vote_count,vote_average,popularity,genres,weighted_rating
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


5 out 10 movies of genres drama. Also three Christopher Nolan Films, **Inception**, **The Dark Knight** and **Interstellar** are part of top 10 list. The list coins a strong bias towards meta data like genres, directors etc. 

Now, building list for particular genres. For that, cutoff is relaxed to **85%** instead of 95. 

In [60]:
temp = m_df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
temp.name = 'genre'
mgen_df = m_df.drop('genres', axis=1).join(temp)

In [61]:
def make_toplist(genre, percentile=0.85):
    df = mgen_df[mgen_df['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    col_list = ['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genre']
    qualified = df[(df['vote_count'] >= m) 
                   & (df['vote_count'].notnull()) 
                   & (df['vote_average'].notnull())][col_list]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['weighted_rating'] = qualified.apply(lambda x: 
                                                   (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), 
                                                   axis=1)
    qualified = qualified.sort_values('weighted_rating', ascending=False).head(250)
    
    return qualified

### Top 10 Romance Movies

In [62]:
make_toplist('Romance').head(10)

,title,year,vote_count,vote_average,popularity,genre,weighted_rating
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,Romance,8.565285
351,Forrest Gump,1994,8147,8,48.3072,Romance,7.971357
876,Vertigo,1958,1162,8,18.2082,Romance,7.811667
40251,Your Name.,2016,1030,8,34.461252,Romance,7.789489
883,Some Like It Hot,1959,835,8,11.8451,Romance,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,Romance,7.744878
19901,Paperman,2012,734,8,7.19863,Romance,7.713951
37863,Sing Street,2016,669,8,10.672862,Romance,7.689483
882,The Apartment,1960,498,8,11.9943,Romance,7.599317
38718,The Handmaiden,2016,453,8,16.727405,Romance,7.566166



#### A bollywood movie Dilwale Dulhania Le Jayenge is getting 1st place

## Content Based Recommendation model

** Limitation of Popularity model ** <br>
It gives the same recommendation to everyone, regardless of the user's personal interest. <br>

_For example:_ If a person who loves romantic movies were to look at Top 15 romantic movies, and he wouldn't probably like most of the listed movies. So, he were to go one step further and look at movie lists by genre, he wouldn't still be getting the interesting recommendations. Let say, he loves *Dilwale Dulhania Le Jayenge*, *My Name is Khan* and *Kabhi Khushi Kabhi Gham*. We can probable assume that he loves the actor Shahrukh Khan and the director Karan Johar. Even if he were to access the top romance list, he wouldn't find these in the list.

Therefore, let's build a model that computes similarity between movies based on certain metrics and suggests movies that are most similar to a particular movie that a user liked. For that we have to consider metadata (or content), hence, it also known as **Content Based Filtering.**

Two Content Based Recommendation is implemented based on different contents:
1. Description Based (content: Movie Overviews and Taglines)
2. Meta Data Based (content : Movie Cast, Crew, Keywords and Genre)

### I am implementing using Description based



** Note **: A small movie data set is used due to limiting computing power available to me. 

In [109]:
small_mdf = pd.read_csv('data/links_small.csv')
small_mdf = small_mdf[small_mdf['tmdbId'].notnull()]['tmdbId'].astype('int')

Before extracting small data set, we need to make sure that the ID column of our main dataframe is clean and of type integer. To do this, let us try to perform an integer conversion of our IDs and if an exception is raised,we will replace the ID with NaN. We will then proceed to drop these rows from our dataframe.

In [64]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [65]:
m_df['id'] = m_df['id'].apply(convert_int)

In [66]:
m_df[m_df['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
19730,- Written by Ã˜rnÃ¥s,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[Carousel Productions, Vision View Entertainme...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,FALSE,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[Aniplex, GoHands, BROSTA TV, Mardock Scramble...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68,"[{'iso_639_1': 'ja', 'name': 'æ—¥æœ¬èªž'}]",Released,NaN,Mardock Scramble: The Third Exhaust,FALSE,7,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[Odyssey Media, Pulser Productions, Rogue Stat...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,FALSE,4.3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [67]:
m_df = m_df.drop([19730, 29503, 35587])
m_df['id'] = m_df['id'].astype('int')

In [68]:
sm_df = m_df[m_df['id'].isin(small_mdf)]
sm_df.shape

(9099, 25)

**9099** movies avaiable in our small movies metadata dataset 

### Description Based Recommendation

In [69]:
sm_df['tagline'] = sm_df['tagline'].fillna('')
sm_df['description'] = sm_df['overview'] + sm_df['tagline']
sm_df['description'] = sm_df['description'].fillna('')

#### Compute TF-IDF matrix 

In [70]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(sm_df['description'])
tfidf_matrix.shape

(9099, 269036)

#### Cosine Similarity

The Cosine Similarity is used to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $

Since the TF-IDF Vectorizer is used, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, sklearn's **linear_kernel** is used instead of cosine_similarities as it's much faster.

In [71]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [72]:
cosine_sim[0]

array([1.        , 0.00680459, 0.        , ..., 0.        , 0.00289024,
       0.        ])

We now have a pairwise cosine similarity matrix for all the movies in our dataset. The next step is to write a function that returns the 50 most similar movies based on the cosine similarity score.

In [73]:
sm_df = sm_df.reset_index()
titles = sm_df['title']
indices = pd.Series(sm_df.index, index=sm_df['title'])

In [74]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Let us now try and get the top 10 recommendations for a few movies.

In [75]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [76]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
8165    Batman: The Dark Knight Returns, Part 1
2696                                        JFK
Name: title, dtype: object

We see that for **The Dark Knight**, our system is able to identify it as a Batman film and subsequently recommend other Batman films as its top recommendations. But unfortunately, that is all this system can do at the moment. This is not of much use to most people as it doesn't take into considerations very important features such as cast, crew, director and genre, which determine the rating and the popularity of a movie. Someone who liked **The Dark Knight** probably likes it more because of Nolan and would hate **Batman Forever** and every other substandard movie in the Batman Franchise.

Therefore, we are going to use much more suggestive metadata than **Overview** and **Tagline**. In the next subsection, Let's build a more sophisticated recommender that takes **genre**, **keywords**, **cast** and **crew** into consideration.

## Collaborative Filtering



** Limitation of content based recommendation**: 
- It is only capable of suggesting movies which are *close* to a certain movie. That is, it is not capable of capturing interest and providing recommendations across genres.

- It doesn't capture the personal intrest and biases of a user. Anyone querying on model for recommendations based on a movie will receive the same recommendations for that movie, regardless of who he is.

Therefore, in this section, we will use a technique called **Collaborative Filtering** to make recommendations to Movie Watchers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.

We will use the **Surprise** library that used extremely powerful algorithms like **Singular Value Decomposition (SVD)** to minimise RMSE (Root Mean Square Error) and give great recommendations.

In [110]:
reader = Reader()

In [111]:
ratings = pd.read_csv('data/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [96]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

In [97]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

CaseInsensitiveDefaultDict(list,
                           {'mae': [0.6892448973302905,
                             0.6924754355137566,
                             0.6922281988093234,
                             0.6956517535515176,
                             0.6854885579732689],
                            'rmse': [0.8940041402383924,
                             0.9007749458410651,
                             0.8965981810762044,
                             0.900286088523052,
                             0.8941489478346697]})

We get a mean **Root Mean Sqaure Error** of 0.8965 which is more than good enough right now. Let us now train on dataset and arrive at predictions.

In [98]:
trainset = data.build_full_trainset()
svd.train(trainset)

Let us pick user 5000 and check the ratings he has given.

In [99]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [100]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.7184215440521635, details={u'was_impossible': False})

For movie with ID 302, we get an estimated prediction of **2.873**. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

## Hybrid Recommender


we build a simple hybrid recommender that brings together techniques we have implemented in the content based and collaborative filter based engines. This is how it will work:

* **Input:** User ID and the Title of a Movie
* **Output:** Similar movies sorted on the basis of expected ratings by that particular user.

In [101]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [102]:
id_map = pd.read_csv('data/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(sm_df[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [103]:
indices_map = id_map.set_index('id')

In [104]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = sm_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [105]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
1180,"The Good, the Bad and the Ugly",2371.0,8.1,1966,429,3.110617
7016,NausicaÃ¤ of the Valley of the Wind,808.0,7.7,1984,81,2.957856
16462,True Grit,1701.0,7.2,2010,44264,2.931916
4101,Double Impact,219.0,5.3,1991,9594,2.796490
3076,Brenda Starr,7.0,5.1,1989,47070,2.749420
3457,Caddyshack,370.0,6.7,1980,11977,2.748869
12165,Far from the Madding Crowd,23.0,6.5,1967,3469,2.738117
3652,Jesus' Son,22.0,6.7,1999,25636,2.731491
6367,Duel at Diablo,22.0,6.3,1966,1403,2.694740
6603,Fire,17.0,6.0,1996,513,2.689907


In [106]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
7016,NausicaÃ¤ of the Valley of the Wind,808.0,7.7,1984,81,3.449868
6367,Duel at Diablo,22.0,6.3,1966,1403,3.338781
1180,"The Good, the Bad and the Ugly",2371.0,8.1,1966,429,3.304091
16462,True Grit,1701.0,7.2,2010,44264,3.269823
5641,Continental Divide,28.0,5.5,1981,26560,3.121401
3529,Shanghai Noon,756.0,6.2,2000,8584,3.052731
3652,Jesus' Son,22.0,6.7,1999,25636,3.000348
12274,Bee Movie,1208.0,5.7,2007,5559,2.983082
15219,Steam of Life,11.0,6.9,2010,52903,2.967189
12165,Far from the Madding Crowd,23.0,6.5,1967,3469,2.949890


We see that for our hybrid recommender, we get different recommendations for different users although the movie is the same. Hence, our recommendations are more personalized and tailored towards particular users.

## Conclusion

In this notebook, we have built 4 different recommendation engines based on different ideas and algorithms. They are as follows:

1. **Simple Recommender:** This system used overall TMDB Vote Count and Vote Averages to build Top Movies Charts, in general and for a specific genre. The IMDB Weighted Rating System was used to calculate ratings on which the sorting was finally performed.
2. **Content Based Recommender:** We built two content based engines; one that took movie overview and taglines as input and the other which took metadata such as cast, crew, genre and keywords to come up with predictions. We also deviced a simple filter to give greater preference to movies with more votes and higher ratings.
3. **Collaborative Filtering:** We used the powerful Surprise Library to build a collaborative filter based on single value decomposition. The RMSE obtained was less than 1 and the engine gave estimated ratings for a given user and movie.
4. **Hybrid Engine:** We brought together ideas from content and collaborative filterting to build an engine that gave movie suggestions to a particular user based on the estimated ratings that it had internally calculated for that user.

## Reference

https://stackabuse.com/creating-a-simple-recommender-system-in-python-using-pandas/

https://www.datacamp.com/community/tutorials/recommender-systems-python


https://www.geeksforgeeks.org/movie-recommendation-based-emotion-python/